<a href="https://colab.research.google.com/github/ag3926/hello_world/blob/master/Instrument_Manholes_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#download and import packages and authorize connection to Google account and open a google spreadsheet
!pip install --upgrade -q gspread
from google.colab import auth
auth.authenticate_user()

import gspread
from oauth2client.client import GoogleCredentials

gc = gspread.authorize(GoogleCredentials.get_application_default())

import pandas
from datascience import *
from pandas import DataFrame

import numpy as np
import math




##THIS IS THE SPREADHSEET INCLUDING THE INSTRUMENT MANHOLES##

spreadsheet = gc.open_by_url("https://docs.google.com/spreadsheets/d/1umFPxBQLV9kEYUwu6VDhYD9o5mLrXjGsaxnR6BjO638")
sheet =  spreadsheet.get_worksheet(0)  # 0 means the first sheet in the file
manholes = pandas.DataFrame(sheet.get_all_records())
manholes=Table.from_df(manholes)





##THIS IS THE SPREADHSEET INCLUDING THE BUILDINGS##

#spreadsheet1 = gc.open_by_url("https://docs.google.com/spreadsheets/d/1TTZzkLpiTsUWJ_1r2NMkc_39H7NqgEvL-cwOARRjdZE/edit?usp=sharing")
sheet1 =  spreadsheet.get_worksheet(1)  # 0 means the first sheet in the file
buildings = pandas.DataFrame(sheet1.get_all_records())
buildings = Table.from_df(buildings)


In [0]:
#creates dictionary; includes building location as the key and distance as the value
loc_dist_dict = {}

#creates function to find the building closest to a specific manhole
def closest(a):
  
  for building_num in range(0, buildings.num_rows):
    distance = np.sqrt((manholes["X Coordinate [ft]"][a] - buildings["X Coordinate [ft]"][building_num])**2 + (manholes["Y Coordinate [ft]"][a] - buildings["Y Coordinate [ft]"][building_num])**2)
    loc_dist_dict[(str(buildings["X Coordinate [ft]"][building_num]) + ", " + (str(buildings["Y Coordinate [ft]"][building_num])))] = distance
  closest = (min(loc_dist_dict.items(), key=lambda x: x[1])[0]) 
  return closest

#computes function for every manhole

closest_building = [closest(manhole_num) for manhole_num in range(0, manholes.num_rows)]
#closest_building
closest_table = Table().with_columns("MHID", manholes["MHID"], "Manhole Location", manholes["Location"], "Manhole X Coordinate [ft]", manholes["X Coordinate [ft]"])
closest_table = closest_table.with_columns("Manhole Y Coordinate [ft]", manholes["Y Coordinate [ft]"])
#print("closest building: ", len(closest_building))
#print("manholes: ", manholes.num_rows)


In [51]:
closest_table

MHID,Manhole Location,Manhole X Coordinate [ft],Manhole Y Coordinate [ft]
15557,FIRST AVE N/O 12 ST,1612481,1113943
15562,42 ST W/O SECOND AVE,1611626,1121696
15568,BROAD ST N/O WATER ST,1610356,1101450
15569,41 ST E/O THIRD AVE,1611073,1121430
15583,41 ST I/S SECOND AVE,1611659,1121460
15612,AVENUE C S/O 18 ST,1614626,1115433
15613,FDR S/O 20 ST,1614314,1115842
15584,AVENUE D S/O 10 ST,1615428,1113223
15578,AVENUE D N/O 10 ST,1615366,1113456
15607,63 ST E/O LEXTINGTON AVE,1610538,1127237


In [0]:
building_coords = [str(buildings["X Coordinate [ft]"][0]) + ", " + str(buildings["Y Coordinate [ft]"][0])]

for n in range(1, buildings.num_rows):
    building_coords.append(str(buildings["X Coordinate [ft]"][n]) + ", " + str(buildings["Y Coordinate [ft]"][n]))

    
    
index_manhole = [building_coords.index(closest_building[0])]

for i in range(1, len(closest_building)):
    index_manhole.append(building_coords.index(closest_building[i])) 
    

In [53]:
closest_table.with_columns("Customer Name", buildings[1][index_manhole], "POE Address", buildings[2][index_manhole], "Customer Address", buildings[0][index_manhole], "Building X Coordinate [ft]", buildings[7][index_manhole], "Building Y Coordinate [ft]", buildings[9][index_manhole])

MHID,Manhole Location,Manhole X Coordinate [ft],Manhole Y Coordinate [ft],Customer Name,POE Address,Customer Address,Building X Coordinate [ft],Building Y Coordinate [ft]
15557,FIRST AVE N/O 12 ST,1612481,1113943,NEW YORK TELEPHONE CO,204-214 SECOND AVE.,204-14 SECOND AVE,1611696,1114097
15562,42 ST W/O SECOND AVE,1611626,1121696,SLG 220 NEWS OWNER LLC,247 E41 ST,223-47 41 ST,1611643,1121506
15568,BROAD ST N/O WATER ST,1610356,1101450,NEW YORK TELEPHONE CO,102 BROAD ST,104 BROAD ST,1610332,1101513
15569,41 ST E/O THIRD AVE,1611073,1121430,ALMA D ASKIN SEYMOUR B ROYAL H AND DAVID DURST,205 E 41ST ST,200 42 ST,1611039,1121440
15583,41 ST I/S SECOND AVE,1611659,1121460,SLG 220 NEWS OWNER LLC,247 E41 ST,223-47 41 ST,1611643,1121506
15612,AVENUE C S/O 18 ST,1614626,1115433,BPP ST OWNER LLC,309 AVE C BLDG.30,309 C AVE,1614614,1115422
15613,FDR S/O 20 ST,1614314,1115842,BPP ST OWNER LLC,325 AVE C BLDG.25,325 C AVE,1614330,1115826
15584,AVENUE D S/O 10 ST,1615428,1113223,BOYS BROTHERHOOD REP OF NEW YORK INC,60 AVENUE D,888 6 ST,1615403,1112349
15578,AVENUE D N/O 10 ST,1615366,1113456,NEW YORK CITY NEWTN CREEK WPCP DEPT WTR RESR,184 AVE D,184 AVENUE D,1615377,1114131
15607,63 ST E/O LEXTINGTON AVE,1610538,1127237,139 EAST 63 STREET INC,139 E63 ST,139 63 ST,1610557,1127260


In [64]:
RTU = {"MHID": manholes["MHID"], "Manhole Location": manholes["Location"], "Manhole X Coordinate [ft]": manholes["X Coordinate [ft]"], "Customer Name": buildings[1][index_manhole], "POE Address": buildings[2][index_manhole], "Customer Address": buildings[0][index_manhole], "Building X Coordinate [ft]": buildings[7][index_manhole], "Building Y Coordinate [ft]": buildings[9][index_manhole]}

df = DataFrame(RTU, columns= ["MHID", "Manhole Location", "Manhole X Coordinate [ft]", "Customer Name", "POE Address", "Customer Address", "Building X Coordinate [ft]", "Building Y Coordinate [ft]"])

export_excel = df.to_excel (r'C:\Users\GonzalezAd\Desktop\RTU_data1.xlsx', index = None, header=True) 

print (df)

      MHID  ... Building Y Coordinate [ft]
0    15557  ...                    1114097
1    15562  ...                    1121506
2    15568  ...                    1101513
3    15569  ...                    1121440
4    15583  ...                    1121506
5    15612  ...                    1115422
6    15613  ...                    1115826
7    15584  ...                    1112349
8    15578  ...                    1114131
9    15607  ...                    1127260
10   15608  ...                    1130703
11   15597  ...                    1129622
12   15588  ...                    1125945
13   15589  ...                    1125953
14   15590  ...                    1125945
15   15585  ...                    1125667
16   15596  ...                    1123586
17   15593  ...                    1115953
18   15594  ...                    1115953
19   15595  ...                    1115873
20   15587  ...                    1114941
21   15577  ...                    1115291
22   15576 